### Use Case 19:

In [ ]:
-- Step 1 - Finding our PII Columns
USE ROLE accountadmin;
USE WAREHOUSE tasty_dev_wh;

SELECT 
    cl.customer_id,
    cl.first_name,
    cl.last_name,
    cl.e_mail,
    cl.phone_number,
    cl.city,
    cl.country
FROM frostbyte_tasty_bytes.raw_customer.customer_loyalty cl;

In [ ]:
--  Step 2 - Creating Tags
USE ROLE accountadmin;

CREATE OR REPLACE TAG frostbyte_tasty_bytes.raw_customer.pii_name_tag
    COMMENT = 'PII Tag for Name Columns';
    
CREATE OR REPLACE TAG frostbyte_tasty_bytes.raw_customer.pii_phone_number_tag
    COMMENT = 'PII Tag for Phone Number Columns';
    
CREATE OR REPLACE TAG frostbyte_tasty_bytes.raw_customer.pii_email_tag
    COMMENT = 'PII Tag for E-mail Columns';

In [ ]:
-- Section 4 - Step 3 - Applying Tags
ALTER TABLE frostbyte_tasty_bytes.raw_customer.customer_loyalty 
    MODIFY COLUMN first_name 
        SET TAG frostbyte_tasty_bytes.raw_customer.pii_name_tag = 'First Name';

ALTER TABLE frostbyte_tasty_bytes.raw_customer.customer_loyalty 
    MODIFY COLUMN last_name 
        SET TAG frostbyte_tasty_bytes.raw_customer.pii_name_tag = 'Last Name';

ALTER TABLE frostbyte_tasty_bytes.raw_customer.customer_loyalty 
    MODIFY COLUMN phone_number 
        SET TAG frostbyte_tasty_bytes.raw_customer.pii_phone_number_tag = 'Phone Number';

ALTER TABLE frostbyte_tasty_bytes.raw_customer.customer_loyalty 
    MODIFY COLUMN e_mail
        SET TAG frostbyte_tasty_bytes.raw_customer.pii_email_tag = 'E-mail Address';

In [ ]:
-- Section 4: Step 4 - Exploring Tags on a Table
SELECT 
    tag_database,
    tag_schema,
    tag_name,
    column_name,
    tag_value 
FROM TABLE(frostbyte_tasty_bytes.information_schema.tag_references_all_columns
    ('frostbyte_tasty_bytes.raw_customer.customer_loyalty','table'));


### Use Case 20:

In [ ]:
--This table will store processed records flagged with the pii_email_tag:
CREATE OR REPLACE TABLE frostbyte_tasty_bytes.raw_customer.email_tagged_audit (
    id STRING,
    e_mail STRING,
    tag_value STRING,
    processed_at TIMESTAMP
);


This stored procedure will:

- Identify which columns have the pii_email_tag.
- Extract relevant email data.
- Log the results.

In [ ]:
CREATE OR REPLACE PROCEDURE frostbyte_tasty_bytes.raw_customer.process_pii_email_tag()
RETURNS STRING
LANGUAGE SQL
AS
$$
BEGIN
    -- Insert data into audit table based on the email tag
    INSERT INTO frostbyte_tasty_bytes.raw_customer.email_tagged_audit (id, e_mail, tag_value, processed_at)
    SELECT 
        cl.id, 
        cl.e_mail, 
        tg.value, 
        CURRENT_TIMESTAMP
    FROM frostbyte_tasty_bytes.raw_customer.customer_loyalty AS cl
    JOIN snowflake.account_usage.tag_references AS tg
        ON tg.object_id = cl.table_id
        AND tg.tag_name = 'frostbyte_tasty_bytes.raw_customer.pii_email_tag'
    WHERE cl.e_mail IS NOT NULL;

    RETURN 'Processed tagged emails successfully';
END;
$$;


This task will execute the stored procedure every 10 minutes.

In [ ]:
CREATE OR REPLACE TASK frostbyte_tasty_bytes.raw_customer.process_pii_email_task
    WAREHOUSE = COMPUTE
    SCHEDULE = '10 MINUTE'
    COMMENT = 'Task to process email-tagged data'
AS
CALL frostbyte_tasty_bytes.raw_customer.process_pii_email_tag();

ALTER TASK frostbyte_tasty_bytes.raw_customer.process_pii_email_task RESUME;

### Use Case 9

#### What This Does:
This section defines **masking policies** in Snowflake, which are security rules that control how sensitive data is displayed to different users based on their **roles**. Masking policies ensure that only authorized users can see full data, while others see a masked version.

#### Key Concepts:
1. **Role-Based Access**: Only users with the `SYSADMIN` or `ACCOUNTADMIN` role can see full data. All other users see masked values.
2. **Conditional Masking**: Each policy checks the current user's role before determining how much data to reveal.
3. **Data Protection**: Sensitive information like names, phone numbers, and email addresses are partially or fully obscured for unauthorized users.



In [ ]:
-- Section 5: Step 1 - Creating Masking Policies
USE ROLE sysadmin;

CREATE OR REPLACE MASKING POLICY frostbyte_tasty_bytes.raw_customer.name_mask AS (val STRING) RETURNS STRING ->
    CASE 
        WHEN CURRENT_ROLE() IN ('SYSADMIN', 'ACCOUNTADMIN') THEN val
    ELSE '**~MASKED~**'
END;

CREATE OR REPLACE MASKING POLICY frostbyte_tasty_bytes.raw_customer.phone_mask AS (val STRING) RETURNS STRING ->
    CASE
        WHEN CURRENT_ROLE() IN ('SYSADMIN', 'ACCOUNTADMIN') THEN val
    ELSE CONCAT(LEFT(val,3), '-***-****')
END;

CREATE OR REPLACE MASKING POLICY frostbyte_tasty_bytes.raw_customer.email_mask AS (val STRING) RETURNS STRING ->
    CASE 
        WHEN CURRENT_ROLE() IN ('SYSADMIN', 'ACCOUNTADMIN') THEN val
    ELSE CONCAT('**~MASKED~**','@', SPLIT_PART(val, '@', -1))
END;
            

This step **links masking policies to data tags**, ensuring that any column with a specific **tag** automatically inherits the corresponding **masking policy**.


In [ ]:
-- Section 5: Step 2 - Applying Masking Policies Tags
USE ROLE accountadmin;

ALTER TAG frostbyte_tasty_bytes.raw_customer.pii_name_tag 
    SET MASKING POLICY frostbyte_tasty_bytes.raw_customer.name_mask;
    
ALTER TAG frostbyte_tasty_bytes.raw_customer.pii_phone_number_tag
    SET MASKING POLICY frostbyte_tasty_bytes.raw_customer.phone_mask;
    
ALTER TAG frostbyte_tasty_bytes.raw_customer.pii_email_tag
    SET MASKING POLICY frostbyte_tasty_bytes.raw_customer.email_mask;

Let's test out the masking policies. 

In [ ]:
USE ROLE tasty_data_engineer;
USE WAREHOUSE tasty_dev_wh;

SELECT 
    cl.customer_id,
    cl.first_name,
    cl.last_name,
    cl.phone_number,
    cl.e_mail,
    cl.city,
    cl.country
FROM frostbyte_tasty_bytes.raw_customer.customer_loyalty cl
WHERE cl.country IN ('United States','Canada','Brazil');


In [ ]:
USE ROLE accountadmin;

SELECT TOP 10
    clm.customer_id,
    clm.first_name,
    clm.last_name,
    clm.phone_number,
    clm.e_mail,
    SUM(clm.total_sales) AS lifetime_sales_usd
FROM frostbyte_tasty_bytes.analytics.customer_loyalty_metrics_v clm
WHERE 1=1
    AND clm.city = 'San Mateo'
GROUP BY clm.customer_id, clm.first_name, clm.last_name, clm.phone_number, clm.e_mail
ORDER BY lifetime_sales_usd DESC;


#### Why This Matters:
- **Enhances Security**: Prevents unauthorized access to personally identifiable information (PII).
- **Regulatory Compliance**: Helps meet data privacy requirements such as **GDPR** and **CCPA**.
- **Flexibility**: Policies can be applied to multiple columns across different tables and adjusted as needed.

### Row-Access Policies in Snowflake

#### What This Does:
This setup **restricts access to specific rows** in a table based on a user's role and their assigned permissions. It ensures that users can only view rows corresponding to the cities they have permission to access.

---

### **Step-by-Step Breakdown:**
#### **Step 1 - Creating a Mapping Table**
- A table (`row_policy_map`) is created to store **role-based city access permissions**.
- Each row links a **user role** to an **allowed city**.

#### **Step 2 - Inserting Mapping Records**
- A sample entry is added:  
  - Users with the role `TASTY_TEST_ROLE` can only see rows where `city = 'Tokyo'`.

#### **Step 3 - Creating a Row Access Policy**
- A **Row Access Policy** (`customer_city_row_policy`) is created to **control row visibility**.
- It grants full access to **administrative roles** (`ACCOUNTADMIN`, `SYSADMIN`, etc.).
- For other users:
  - The policy checks if their **current role** exists in the `row_policy_map` table.
  - If a role has permissions for a specific city, the user sees only those rows.

#### **Step 4 - Applying the Policy to a Table**
- The **Row Access Policy** is applied to the `customer_loyalty` table on the `city` column.
- Now, when a user queries `customer_loyalty`, they only see rows where:
  - Their role is explicitly allowed to view the city.
  - Or they belong to an admin-level role.

In [ ]:
-- Step 1 - Creating a Mapping Table
USE ROLE accountadmin;

CREATE OR REPLACE TABLE frostbyte_tasty_bytes.public.row_policy_map
    (role STRING, city_permissions STRING);

    
-- Step 2 - Inserting Mapping Records
INSERT INTO frostbyte_tasty_bytes.public.row_policy_map
    VALUES ('TASTY_DATA_ENGINEER','Tokyo');
    

-- Step 3 - Creating a Row Access Policy
CREATE OR REPLACE ROW ACCESS POLICY frostbyte_tasty_bytes.public.customer_city_row_policy
    AS (city STRING) RETURNS BOOLEAN ->
       CURRENT_ROLE() IN 
       (
           'ACCOUNTADMIN','SYSADMIN', 'TASTY_ADMIN', 
           'TASTY_DATA_APP','TASTY_BI','TASTY_DATA_SCIENTIST','TASTY_DEV'
       ) 
        OR EXISTS 
            (
            SELECT rp.role 
                FROM frostbyte_tasty_bytes.public.row_policy_map rp
            WHERE 1=1
                AND rp.role = CURRENT_ROLE()
                AND rp.city_permissions = city
            );

            
-- Step 4 - Applying a Row Access Policy to a Table
ALTER TABLE frostbyte_tasty_bytes.raw_customer.customer_loyalty
    ADD ROW ACCESS POLICY frostbyte_tasty_bytes.public.customer_city_row_policy ON (city);


Let's test out the row-level policies.

In [ ]:
-- Step 5 - Testing our Row Access Policy in a Non-Privileged Role
USE ROLE tasty_data_engineer;

SELECT 
    cl.customer_id,
    cl.first_name,
    cl.last_name,
    cl.city,
    cl.marital_status,
    cl.age AS age
FROM frostbyte_tasty_bytes.raw_customer.customer_loyalty cl
GROUP BY cl.customer_id, cl.first_name, cl.last_name, cl.city, cl.marital_status, age;



In [ ]:
-- Step 7 - Testing our Row Access Policy in a Privileged Role
USE ROLE sysadmin;

SELECT 
    cl.customer_id,
    cl.first_name,
    cl.last_name,
    cl.city,
    cl.marital_status,
    cl.age
FROM frostbyte_tasty_bytes.raw_customer.customer_loyalty cl
GROUP BY cl.customer_id, cl.first_name, cl.last_name, cl.city, cl.marital_status, age;




### **Why This Matters:**
- **Data Security & Compliance**: Ensures that users only access data they are authorized to view.
- **Role-Based Row Filtering**: Different users see different data dynamically.
- **Scalability**: New access rules can be added to `row_policy_map` without modifying the policy.

✅ Snowflake maintains security policies even when data is shared, while our Spark-based competitors require manual re-implementation.


### Snowflake Differential Privacy

Demonstrates differential privacy techniques while showcasing advanced governance features.

Differentiators:
- Snowflake provides native privacy policy enforcement, role-based access control, 
and automatic noise addition at the query level. 
- Unlike other platforms that require additional tools, Snowflake integrates privacy protection directly into the SQL layer, ensuring **seamless** governance.

-- 

Step 1

Snowflake allows defining a Privacy Policy to enforce role-based privacy constraints dynamically.
- No need for external privacy frameworks—built-in differential privacy support.
- Automatic privacy budget management at the database level, reducing compliance overhead.
- Fine-grained role-based access control (RBAC) for privacy-aware query execution.


In [ ]:

-- =============================================================
-- 1. Privacy Policy Implementation 
-- =============================================================
-- 
USE ROLE ACCOUNTADMIN;

CREATE OR REPLACE PRIVACY POLICY FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.privacy_policy AS () RETURNS privacy_budget ->
  CASE
    WHEN CURRENT_ROLE() = 'ACCOUNTADMIN' THEN no_privacy_policy()
    WHEN CURRENT_ROLE() IN ('TASTY_BI')
      THEN privacy_budget(budget_name => 'analysts')
    ELSE privacy_budget(budget_name => 'default')
END;

-- Assign the privacy policy to the table.
ALTER TABLE FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY
ADD PRIVACY POLICY FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.privacy_policy ENTITY KEY (customer_id);


In [ ]:
-- =============================================================
-- 2. Privacy Domains Enforcement 
-- =============================================================
-- Snowflake allows explicit privacy domain enforcement, ensuring category-based protection.
-- - Unlike alternative solutions requiring manual masking, Snowflake automates category-based privacy enforcement.
-- - The system prevents over-querying individual attributes, mitigating deanonymization risks.


ALTER TABLE FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY ALTER (
COLUMN gender SET PRIVACY DOMAIN IN ('female', 'male', 'undisclosed'),
COLUMN marital_status SET PRIVACY DOMAIN IN ('single', 'married', 'undisclosed'),
COLUMN age SET PRIVACY DOMAIN BETWEEN (0,90)
);


In [ ]:
-- =============================================================
-- 3. Role-Based Query Access Demonstration
-- =============================================================
-- Snowflake ensures role-based privacy enforcement, restricting access dynamically.
-- - Unlike alternatives requiring manual data redaction, Snowflake applies automatic query adjustments.
-- - Noise injection ensures that aggregations remain privacy-safe.

USE ROLE ACCOUNTADMIN;
SELECT * FROM FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY;


-- =============================================================
-- 4. Differential Privacy in Aggregations
-- =============================================================
-- Snowflake allows controlled privacy-preserving aggregates, adding noise dynamically.
-- - Unlike basic role-based masking, Snowflake adds quantifiable differential privacy.
-- - Ensures that repeated queries do not leak sensitive individual data.





In [ ]:
USE ROLE TASTY_BI;
SELECT * FROM FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY;


In [ ]:
USE ROLE ACCOUNTADMIN;
SELECT COUNT(DISTINCT customer_id)
  FROM FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY
  WHERE age > 50;

In [ ]:
USE ROLE TASTY_BI;
USE WAREHOUSE TASTY_BI_WH;
SELECT COUNT(DISTINCT customer_id)
  FROM FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY
  WHERE age > 50;


In [ ]:
-- =============================================================
-- 5. Privacy-Aware Confidence Intervals for Queries
-- =============================================================
-- Snowflake allows users to retrieve noise bounds to understand privacy-preserving error margins.
-- - Unlike alternatives that require manual statistical analysis, Snowflake provides built-in uncertainty bounds.
-- - Ensures analysts receive privacy-safe insights without exposing raw values.

USE ROLE TASTY_BI;
SELECT COUNT(DISTINCT customer_id) as c,
  DP_INTERVAL_LOW(c) as LOW,
  DP_INTERVAL_HIGH(c) as HIGH
  FROM FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY
  WHERE age > 50;


### Why this matters


- ✅ **Built-in differential privacy enforcement** at the SQL level.
- ✅ **Role-based privacy policies** without needing external tools.
- ✅ **Category-based privacy domains** prevent data re-identification.
- ✅ **Privacy-safe aggregation** with noise-based protections.
- ✅ **Dynamic policy removal & updates** for compliance adaptability.

In [ ]:
/**********************************************************************/
/*------               Quickstart Reset Scripts                 ------*/
/*------   These can be ran to reset your account to a state    ------*/
/*----- that will allow you to run through this Quickstart again -----*/
/**********************************************************************/

USE ROLE accountadmin;

ALTER TAG frostbyte_tasty_bytes.raw_customer.pii_name_tag UNSET MASKING POLICY frostbyte_tasty_bytes.raw_customer.name_mask;
ALTER TAG frostbyte_tasty_bytes.raw_customer.pii_phone_number_tag UNSET MASKING POLICY frostbyte_tasty_bytes.raw_customer.phone_mask;
ALTER TAG frostbyte_tasty_bytes.raw_customer.pii_email_tag UNSET MASKING POLICY frostbyte_tasty_bytes.raw_customer.email_mask;

DROP TAG IF EXISTS frostbyte_tasty_bytes.raw_customer.pii_name_tag;
DROP TAG IF EXISTS frostbyte_tasty_bytes.raw_customer.pii_phone_number_tag;
DROP TAG IF EXISTS frostbyte_tasty_bytes.raw_customer.pii_email_tag;

ALTER TABLE frostbyte_tasty_bytes.raw_customer.customer_loyalty
DROP ROW ACCESS POLICY frostbyte_tasty_bytes.public.customer_city_row_policy;

DROP TABLE IF EXISTS frostbyte_tasty_bytes.public.row_policy_map;
ALTER TABLE FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY
  DROP PRIVACY POLICY FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.privacy_policy;


In [ ]:
ALTER TABLE FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY
  DROP PRIVACY POLICY FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.privacy_policy;